# Argone Leadership Computing Facility Artificial Intelligence Training Program
## Cesar Francisco Quinones-Martinez (cquinones24)
Session 1: Intro to Artificial Intelligence on Supercomputers, 2024-02-09

In this Notebook, I apply the information given by Dr. Huihuo Zheng about linear regression to work on the Homework assignment. I first pass the iterative version of linear regression to verify that it functions in this copy. Afterwards, I will include the changes that were asked for in the Homework.

In [2]:
# Importing the relevant libraries and necessary real estate data:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipydis
import time

! [ -e ./slimmed_realestate_data.csv ] || wget https://raw.githubusercontent.com/argonne-lcf/ai-science-training-series/main/01_intro_AI_on_Supercomputer/slimmed_realestate_data.csv
data = pd.read_csv('slimmed_realestate_data.csv')
print(data.columns)

Index(['Unnamed: 0', 'SalePrice', 'GrLivArea'], dtype='object')


In [ ]:
# Defining neccessary variables for the linear model:
# y = m*x + b
m = 5
b = 1000

# Learning rates for m and b and loss history to observe how the model closes in on the answer:
learning_rate_m = 1e-7
learning_rate_b = 1e-1

loss_history = []

In [ ]:
# Converting data to numpy arrays:
data_x = data['GrLivArea'].to_numpy()
data_y = data['SalePrice'].to_numpy()
loop_N = 30